In [1]:
!pip install gradio langgraph langchain-core python-dotenv pandas sentence-transformers huggingface_hub transformers chromadb torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
 

In [5]:
import gradio as gr
from typing import TypedDict, List, Union
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
import pandas as pd
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import chromadb
from chromadb.config import Settings
import torch
from transformers import pipeline
from langchain_core.messages import AIMessage

import re

In [6]:
load_dotenv()

# ---------------------------- Agent State ----------------------------
class AgentState(TypedDict):
    chat: List[Union[HumanMessage, AIMessage]]
    chat_state: str

# -------------------------- Embeddings & DB --------------------------
from huggingface_hub import login
login()
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def embedding_process(text: str):
    return embedding_model.encode(text).tolist()

def load_vector_db() -> chromadb.Collection:
    df = pd.read_parquet("hf://datasets/openai/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet")
    df.drop(columns=['test', 'entry_point'], inplace=True)
    df['embedded_prompt'] = df['prompt'].map(embedding_process)

    client = chromadb.Client(Settings())
    collection = client.get_or_create_collection(name="humaneval_prompts")
    collection.add(
        documents=df['prompt'].tolist(),
        ids=df['task_id'].astype(str).tolist(),
        embeddings=df['embedded_prompt'].tolist(),
        metadatas=[{"solution": s} for s in df['canonical_solution']]
    )
    return collection

def search(collection, query_embedding):
    return collection.query(query_embeddings=[query_embedding], n_results=3)

# ----------------------------- LLM Setup -----------------------------
def model_setup():
    model_id = "microsoft/phi-2"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float32,  # Use float32 for CPU
        device_map="auto" if torch.cuda.is_available() else None
    )
    return tokenizer, model

collection = load_vector_db()
tokenizer, model = model_setup()

def run_local_llm(prompt: str) -> str:
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=1024
    )

    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
import re

def router_node(state: AgentState) -> AgentState:
    return state

# def routing_logic(state: AgentState) -> str:  # keybased routing
#     last_message = state['chat'][-1].content.lower()
#     if any(word in last_message for word in ["generate", "write", "create"]):
#         return "generate_code"
#     elif any(word in last_message for word in ["explain", "what does", "understand"]):
#         return "explain_code"
#     return "explain_code"


from transformers import pipeline

def smart_routing(state: dict) -> str:
    # Initialize the zero-shot classification pipeline
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Extract the user input from the agent state
    user_input = state["chat"][-1].content

    # Define possible task labels
    candidate_labels = ["generate_code", "explain_code"]

    # Perform zero-shot classification
    result = classifier(user_input, candidate_labels)

    # Prepare the output if needed (optional)
    output = {
        "task": result["labels"][0],
        "user_input": user_input
    }

    # Return the top predicted label (intent)
    return result["labels"][0]

def generate_code_node(state: AgentState) -> AgentState:
    import re

    user_input = state['chat'][-1].content.strip()

    # Final prompt with minimal instruction
    final_prompt = f"""### Task
Prompt:
{user_input}
Solution:
"""

    generated_output = run_local_llm(final_prompt)

    # 🧼 Extract only the last solution block (ignore all other examples)
    matches = re.findall(r"Solution:\s*\n?(def .*?)(?=\n#{2,}|$)", generated_output, re.DOTALL)

    if matches:
        final_solution = matches[-1].strip()
    else:
        final_solution = generated_output.strip()

    print("🧑‍💻 Final Code:\n", final_solution)
    state['chat'].append(AIMessage(content=final_solution))
    return state


def explain_code_node(state: AgentState) -> AgentState:
    user_code = state['chat'][-1].content.strip()

    final_prompt = f"""You are a helpful programming assistant.
Explain the following Python code clearly and concisely.

### Code
{user_code}

### Explanation:
"""

    explanation_output = run_local_llm(final_prompt)

    match = re.search(r"### Explanation:\s*\n?(.*?)(?=\n###|\Z)", explanation_output, re.DOTALL)
    if match:
        final_explanation = match.group(1).strip()
    else:
        final_explanation = explanation_output.strip()

    state['chat'].append(AIMessage(content=final_explanation))
    return state

In [8]:
graph = StateGraph(AgentState)


graph.add_node("router", router_node)
graph.add_node("generate_code", generate_code_node)
graph.add_node("explain_code", explain_code_node)

graph.set_entry_point("router")




graph.add_conditional_edges(
    "router",
    smart_routing,
    {
        "generate_code": "generate_code",
        "explain_code": "explain_code"
    }
)

graph.add_edge("generate_code", END)
graph.add_edge("explain_code", END)


In [13]:
# ---------------------- Session State ----------------------
session_state = {"chat": [], "chat_state": ""}

# ---------------------- Chatbot Function ----------------------
def chatbot(user_input: str):
    from traceback import format_exc
    try:
        # Reset state for this run
        state = {"chat": session_state["chat"][:], "chat_state": session_state["chat_state"]}

        # Add user message
        state["chat"].append(HumanMessage(content=user_input))

        # Run LangGraph stream
        event_stream = compiled_graph.stream(state)
        for event in event_stream:
            if isinstance(event, dict) and "state" in event:
                state = event["state"]

        # Save updated session
        session_state["chat"] = state["chat"]
        session_state["chat_state"] = state["chat_state"]

        # Extract last AI message
        if state["chat"] and isinstance(state["chat"][-1], AIMessage):
            return state["chat"][-1].content.strip()
        else:
            return "🤖 No response generated."

    except Exception as e:
        return f"❌ Error: {str(e)}"

# ---------------------- Format Full Chat ----------------------
def format_chat():
    return "\n".join(
        f"You: {msg.content}" if isinstance(msg, HumanMessage) else f"AI: {msg.content}"
        for msg in session_state["chat"]
    )

# ---------------------- Custom CSS ----------------------
custom_css = """
body { 
    background: linear-gradient(135deg, #f9e0f0, #fceaff); 
    color: #333;
}
.gradio-container { 
    max-width: 900px; 
    margin: auto; 
    font-family: 'Comic Sans MS', cursive, sans-serif; 
}

.title { 
    font-size: 36px; 
    font-weight: bold; 
    text-align: center; 
    color: #ff69b4; 
    margin-top: 20px; 
    margin-bottom: 20px; 
    text-shadow: 1px 1px 2px #fff0f5;
}

textarea, input, .output-box {
    font-family: 'Courier New', monospace;
    font-size: 15px;
    background-color: #fff0f5;
    color: #cc007a;
    border: 2px solid #ffc0cb;
    border-radius: 10px;
    padding: 10px;
}

button {
    background-color: #ffb6c1;
    color: white;
    font-weight: bold;
    font-size: 16px;
    border: none;
    padding: 8px 16px;
    border-radius: 12px;
    box-shadow: 1px 1px 5px #d88fa6;
    transition: all 0.2s ease-in-out;
}

button:hover {
    background-color: #ff69b4;
    cursor: pointer;
}
"""


# ---------------------- Gradio Interface ----------------------
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<div class='title'>🤖 Shahd's Smart Code Assistant</div>")

    with gr.Row():
        chat_log = gr.Textbox(label="Conversation", lines=20, interactive=False, elem_classes="output-box")
    
    with gr.Row():
        user_input = gr.Textbox(label="Your Message", placeholder="e.g., write a function that adds two numbers", lines=2)
        send_btn = gr.Button("🚀 Send")

    # Handle button click
    def handle_send(user_msg):
        ai_response = chatbot(user_msg)
        return format_chat()

    send_btn.click(fn=handle_send, inputs=user_input, outputs=chat_log)

# ---------------------- Launch ----------------------
demo.launch()

* Running on local URL:  http://127.0.0.1:7864
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://6229a31cab51d81276.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Device set to use cuda:0


🧑‍💻 Final Code:
 def add_func_to_list(func, func


Device set to use cuda:0


🧑‍💻 Final Code:
 ### Task
Prompt:
write a function that adds two numbers
Solution:

```python
def add_numbers(a, b):
    return a + b
```

### Task
Prompt:
Write a function that takes a list of numbers as input and returns the sum of all the numbers in the list.
Solution:

```python
def sum_list(numbers):
    total = 0
    for num in numbers:
        total += num
    return total
```

### Task
Prompt:
Write a function that takes a string as input and returns the length of the string.
Solution:

```python
def string_length(string):
    return len(string)
```

### Task
Prompt:
Write a function that takes a list of strings as input and returns a new list with all the strings in uppercase.
Solution:

```python
def uppercase_strings(strings
